<a href="https://colab.research.google.com/github/google-research/protein-ligand-binding-free-energy-calculations/blob/matteo-dev/colab_tutorial/abfe_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [12]:
#@title Install Gromacs and Gromacs Python API
!wget https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz -O /tmp/gromacs.tar.gz
!tar zxf /tmp/gromacs.tar.gz
!sudo rm -rf /usr/local/gromacs
!sudo mv gromacs-avx /usr/local/gromacs
!rm /tmp/gromacs.tar.gz

!pip3 install --upgrade pip
!pip3 install setuptools wheel cmake pybind11 py3DMol
# This ensure we always install the latest PMX version from GitHub.
!if [ -d /usr/local/lib/python3.7/dist-packages/pmx/ ]; then yes Y | pip3 uninstall pmx; fi
!pip3 install git+https://github.com/deGrootLab/pmx.git@abfe_dev
!gmxapi_ROOT=/usr/local/gromacs/ pip3 install --no-cache-dir gmxapi

# Add path to where pmx gets installed.
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-11-15 01:47:34--  https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23920923 (23M) [application/x-gzip]
Saving to: ‘/tmp/gromacs.tar.gz’

/tmp/gromacs.tar.gz 100%[===================>]  22.81M  11.3MB/s    in 2.0s    

2022-11-15 01:47:36 (11.3 MB/s) - ‘/tmp/gromacs.tar.gz’ saved [23920923/23920923]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: pmx 0+untagged.886.g36e3fa1
Uninstalling pmx-0+untagged.886.g36e3fa1:
  Would remove:
    /usr/local/bin/pmx
    /usr/local/lib/python3.7/dist-packages/pmx-0+untagged.886.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#@title Set Level for Logging
#@markdown Sets the python logging level. This is useful for debugging.
import logging

log_level = 'INFO' #@param ['DEBUG', 'INFO', 'ERROR']

_log_level = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'ERROR': logging.ERROR
}

logging.basicConfig(level=_log_level[log_level])

In [14]:
#@title [Please Rerun after Restart] Create a Session
#@markdown Please rerun this cell every time when you restart the runtime.
#@markdown A new session ID will be created for a new colab runtime session.
#@markdown Run this cell again will refresh the session ID.

import os
import time
import gmxapi


# Sets a session ID for this session.
_SESSION_ID = str(int(time.time())) #@param

# Monkey patch this session ID into the gmxapi.operation.ResourceManager.
# Hack. Do not use in the user code.
# This is temporarily added to allow users to start from a new set of working
# directories for all the mdrun launches. GMX API doesn't expose the
# underlying resource manager or context yet.
def operation_id(self):
  return f"{self._base_operation_id}_{_SESSION_ID}"
gmxapi.operation.ResourceManager.operation_id = property(operation_id)


def get_work_dir(md_output):
  values = [v for v in md_output.values()]
  return values[0].result()


def tail_log(md_output, output_file_base, last_n_lines=6):
  work_dir = get_work_dir(md_output)

  with open(os.path.join(work_dir, output_file_base + '.log'), 'r') as log:
    for line in log.readlines()[-last_n_lines:]:
      print(line)

print("Session ID: ", _SESSION_ID)

Session ID:  1668476992


# ABFE Workflow
-----

In [ ]:
import numpy as np
import shutil
from tqdm import tqdm
import pmx
from pmx.AbsoluteDG import AbsoluteDG

In [15]:
#@title Define some helper functions

def mdrun(tpr, pmegpu=True, nsteps=None, transition=False):
  """Wrapper for gmxapi.mdrun with predefined scenarios optimized for different 
  types of simulations.

  Args:
    tpr (str): path to TPR file.
    pmegpu (bool): whether to run PME calculations on the GPU. Default is True.
      Note this is not possible with certain integrators.
    nsteps (int): number of integration steps to run, overwrites nsteps defined
      by the MDP file used to generate the TPR. Default is None (do not modify 
      TPR).
    transition (bool): whether we are running a non-equilibrium transition. 
      Default is False.

  Returns:
    object: StandardOperationHandle returned by gmxapi.mdrun.
  """
  # Load TPR file.
  input_tpr = gmxapi.read_tpr(tpr)
  if nsteps is not None:
    input_tpr = gmxapi.modify_input(input=input_tpr, 
                                    parameters={'nsteps': nsteps})    

  # Get path to TPR.
  path = "/".join(tpr.split('/')[:-1])
  
  # Specify command line flags and arguments to be passed to mdrun.
  # Note: energy minimization cannot run PME on GPU.
  if pmegpu:
    _pme = 'gpu'
    _pmefft = 'gpu'
    _bonded = 'gpu'
  else:
    _pme = 'auto'
    _pmefft = 'auto'
    _bonded = 'auto'

  # If we are running a non-equilibrium transition, we want to get the dhdl
  # file and keep only that. We then use the presence of dhdl_{n}.xvg to 
  # determine if the simulation has been ran already.
  if transition:
    n = int(tpr.split('/')[-1].split('.')[0].replace('ti', ''))
    md = gmxapi.mdrun(input_tpr,
                      runtime_args={'-nb': 'gpu', 
                                    '-pme': _pme, 
                                    '-pmefft': _pmefft, 
                                    '-bonded': _bonded,
                                    '-x': f'{path}/traj_{n}.xtc',
                                    '-o': f'{path}/traj_{n}.trr',
                                    '-c': f'{path}/confout_{n}.gro',
                                    '-e': f'{path}/ener_{n}.edr', 
                                    '-g': f'{path}/md_{n}.log',
                                    '-cpo': f'{path}/state_{n}.cpt',
                                    '-dhdl': f'{path}/dhdl_{n}.xvg'
                                    }
                      )
  else:
    md = gmxapi.mdrun(input_tpr,
                      runtime_args={'-nb': 'gpu', 
                                    '-pme': _pme, 
                                    '-pmefft': _pmefft, 
                                    '-bonded': _bonded,
                                    '-x': f'{path}/traj.xtc',
                                    '-o': f'{path}/traj.trr',
                                    '-c': f'{path}/confout.gro',
                                    '-e': f'{path}/ener.edr', 
                                    '-g': f'{path}/md.log',
                                    '-cpo': f'{path}/state.cpt'
                                    }
                      )

  # Run the simulation.
  md.run()

  # If transition, clean up files we won't need.
  if transition:
    for f in [f'{path}/traj_{n}.xtc', 
              f'{path}/traj_{n}.trr', 
              f'{path}/confout_{n}.gro', 
              f'{path}/ener_{n}.edr', 
              f'{path}/md_{n}.log',  
              f'{path}/state_{n}.cpt']:
      if os.path.isfile(f):
        os.remove(f)

  return md


def mdrun_completed(tpr: str, transition: bool = False) -> bool:
  """Checks whether energy minimization completed successfully.
  
  Args:
    tpr (str): path to TPR file.
    transition (bool): whether we are checking completion for a non-equilibrium 
      transition. Default is False.

  Returns:
    bool: whether the TPR has been run successfully.
  """
  # If we're chekcing the completion of a non-eq transition, we check that (i)
  # the right dhdl.xvg file exists, and that (ii) it contains info up to the 
  # end of the transition. Otherwise, we assume it crashed and needs to be
  # completed.
  if transition:
    # Get the index of the transition.
    n = int(tpr.split('/')[-1].split('.')[0].replace('ti', ''))
    # Get dhdl file path.
    dhdl = "/".join(tpr.split("/")[:-1]) + f"/dhdl_{n}.xvg"
    # If dhdl file exists, check it's complete.
    if os.path.isfile(dhdl):
      input_tpr = gmxapi.read_tpr(tpr)
      nsteps = input_tpr.output.parameters.result()['nsteps']
      dt = input_tpr.output.parameters.result()['dt']
      expected_final_time = nsteps * dt
      
      with open(dhdl, 'r') as f:
        lines = f.readlines()
      
      try:
        actual_final_time = float(lines[-1].split()[0])
        if expected_final_time - actual_final_time < 1e-6:
          return True
      except:
        return False
      
    return False
  else:
    # (maldeghi): IIRC Gromacs would output the GRO file only for mdruns that
    # did not crash/errored. We can make this stricted by checking the log
    # file too.
    gro = "/".join(tpr.split("/")[:-1]) + "/confout.gro"
    if os.path.isfile(gro):
      return True
    else:
      return False


def tail(fname, n):
  with open(fname) as f:
    for line in (f.readlines() [-n:]):
      print(line, end ='')

In [17]:
#@title Copy input files

WORKDIR = f"/content/pmxrun_{_SESSION_ID}"  #@param {type:"string"}
OVERWRITE_WORKDIR = True  #@param {type:"boolean"}

#@markdown Whether to run very short simulations for testing. Otherwise, we run 
#@markdown the full set of calculations, which takes several hours.
SHORT_SIMS = True  #@param {type:"boolean"}

if os.path.isdir(WORKDIR) and OVERWRITE_WORKDIR:
  shutil.rmtree(WORKDIR)

if not os.path.isdir(WORKDIR):
  os.mkdir(WORKDIR)

  pmx_path = pmx.__file__.replace('/__init__.py', '')

  # Copy topology and structure files for protein and ligand.
  shutil.copytree(f'{pmx_path}/abfe_scripts/struct_top/', f'{WORKDIR}/struct_top/')

  # Copy MDP files (i.e. Gromacs config files).
  shutil.copytree(f'{pmx_path}/abfe_scripts/mdppath/', f'{WORKDIR}/mdppath/')

# We also shorten the alchemical transitions to speed things up.
# Note we cannot just change nsteps in mdrun here because we need to adjust the
# lambda schedule too.
! if [ $SHORT_SIMS == "True" ]; then sed -i 's/nsteps.*/nsteps = 25000/g' $WORKDIR/mdppath/ti_l*mdp; fi
! if [ $SHORT_SIMS == "True" ]; then sed -i 's/delta-lambda.*/delta-lambda = 4e-5/g' $WORKDIR/mdppath/ti_l0.mdp; fi  # 1/25,000 = 4e-5
! if [ $SHORT_SIMS == "True" ]; then sed -i 's/delta-lambda.*/delta-lambda = -4e-5/g' $WORKDIR/mdppath/ti_l1.mdp; fi

# Show contents of our working directory.
!echo "> ls {WORKDIR}"
!ls -l {WORKDIR}

> ls /content/pmxrun_1668476992
total 8
drwxr-xr-x 2 root root 4096 Nov 15 01:50 mdppath
drwxr-xr-x 4 root root 4096 Nov 15 01:48 struct_top


## 1.&nbsp; Input Files Preparation

In [20]:
#@title 1.1&nbsp; Setup folder structure

# Initialize the free energy environment object. It will store the main 
# parameters for the calculations.
fe = AbsoluteDG(ligList=['lysozyme_benzene'], 
                apoCase='lysozyme_apo', 
                bDSSB=False,
                gmxexec='/usr/local/gromacs/bin/gmx')

# Set the workpath in which simulation input files will be created.
fe.workPath = f'{WORKDIR}/lysopath'
# Set the path to the MDP files.
fe.mdpPath = f'{WORKDIR}/mdppath'
#@markdown Set the number of replicas (i.e., number of equilibrium simulations per state).
fe.replicas = 2  #@param {type:"integer"}
# Provide the path to the structures and topologies.
fe.structTopPath = f'{WORKDIR}/struct_top'

# Prepare the directory structure with all simulations steps required.
fe.simTypes = ['em',  # Energy minimization.
               'eq_posre',  # Equilibrium sim with position restraints.
               'eq',  # Equilibrium simulation.
               'transitions']  # Alchemical, non-equilibrium simulations.

fe.prepareFreeEnergyDir()


---------------------
Summary of the setup:
---------------------

   workpath: /content/pmxrun_1668476992/lysopath
   mdp path: /content/pmxrun_1668476992/mdppath
   # ligands: 1
   ligands:
        lysozyme_benzene
   apo state: lysozyme_apo

---------------------
Directory structure:
---------------------

/content/pmxrun_1668476992/lysopath/
|
|--ligX
|--|--water
|--|--|--stateA
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--protein
|--|--|--stateA
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--strTopFolder
|--lig..

DONE


In [21]:
#@title 1.2&nbsp; Assemble simulation systems

# Assemble the systems: build Gromacs structure and topology for the 
# ligand+water and ligand+protein+water systems.
fe.assemble_systems()

# Define the simulation boxes, fill them with water molecules, and add ions to 
# neutralize the system and reach desired NaCl concentration (0.15 M by default).
fe.boxWaterIons()

----------------------
Assembling the systems
----------------------
Order: ligand-protein-other-water
--- Assembling structures: lysozyme_benzene ---
--- Assembling topologies: lysozyme_benzene ---
----------------
Box, water, ions
----------------


In [22]:
# Check the files present now. This is the content of root dir of the 
# calculation for this protein-ligand pair. The following ls calls show 
# the content of some subfolders for the complex simulations, 
# stateA (coupled ligand). The other folders have the same structure/files but 
# for the ligand in water simulations and for stateB too.
!echo "> ls {WORKDIR}/lysopath/lysozyme_benzene/"
!ls {WORKDIR}/lysopath/lysozyme_benzene/
!echo ""
!echo "> ls {WORKDIR}/lysopath/lysozyme_benzene/protein"
!ls {WORKDIR}/lysopath/lysozyme_benzene/protein/
!echo ""
!echo "> ls {WORKDIR}/lysopath/lysozyme_benzene/protein/stateA"
!ls {WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/
!echo ""
!echo "> ls {WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/"
!ls {WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/

> ls /content/pmxrun_1668476992/lysopath/lysozyme_benzene/
protein  strTopFolder_apo  strTopFolder_holo  water

> ls /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein
stateA	stateB

> ls /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA
box.pdb   mdout.mdp  run2	 topol.top  tpr.tpr
ions.pdb  run1	     system.pdb  top.top    water.pdb

> ls /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run1/
em  eq	eq_posre  transitions


## 2&nbsp; Energy Minimization

In [23]:
#@title 2.1&nbsp; Prepare TPR files

# Call grompp and create TPR files for all systems and repeats.
tpr_files = fe.prepare_simulation(simType='em')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: em
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run2/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run1/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run2/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run1/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run2/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run1/em/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run2/em/tpr.tpr


In [24]:
#@title 2.2&nbsp; Run minimizations

if SHORT_SIMS:
  nsteps = 500
else:
  nsteps = None  # i.e. use what's in the MDP files

# Read the TPR files and run all minimizations.
for tpr_file in tqdm(tpr_files):

  # If minimization has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps if needed)
  md = mdrun(tpr_file, pmegpu=False, nsteps=nsteps)

100%|██████████| 8/8 [02:01<00:00, 15.24s/it]


## 3.&nbsp; Equilibrium Simulations

First, we run a short MD simulation with position restraints to equilibrate the solvent and ions around the protein/ligand. Then, we run tha actual production simulation.

In [25]:
#@title 3.1&nbsp; Prepare TPR files for short equilibration

tpr_files = fe.prepare_simulation(simType='eq_posre', prevSim='em')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: eq_posre
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run2/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run1/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run2/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run1/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run2/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run1/eq_posre/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run2/eq_posre/tpr.tpr


In [26]:
#@title 3.2&nbsp; Run short equilibration

if SHORT_SIMS:
  nsteps = 5_000  # 5,000 x 2 fs = 10 ps
else:
  nsteps = None  # i.e. use what's in the MDP files

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue
  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, nsteps=nsteps)

100%|██████████| 8/8 [01:48<00:00, 13.58s/it]


In [29]:
# Show output files for one run.
print("Path to mdrun output files:", os.path.dirname(tpr_files[0]))
print("Output files:", os.listdir("/".join(tpr_files[0].split("/")[:-1])))
print()

# Tail log file to see performance.
tail(tpr_files[0].replace('tpr.tpr', 'md.log'), n=5)

Path to mdrun output files: /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/eq_posre
Output files: ['tpr.tpr', 'traj.trr', 'md.log', 'confout.gro', 'state.cpt', 'ener.edr', 'mdout.mdp']

       Time:        6.019        3.010      200.0
                 (ns/day)    (hour/ns)
Performance:      287.132        0.084
Finished mdrun on rank 0 Tue Nov 15 01:58:35 2022



In [30]:
#@title 3.3&nbsp; Prepare TPR files for equilibrium simulations

tpr_files = fe.prepare_simulation(simType='eq', prevSim='eq_posre')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: eq
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run2/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run1/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run2/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run1/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run2/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run1/eq/tpr.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run2/eq/tpr.tpr


In [31]:
#@title 3.4&nbsp; Run equilibrium simulations

if SHORT_SIMS:
  nsteps = 25_000  # 25,000 x 2 fs = 50 ps
else:
  nsteps = None  # i.e. use what's in the MDP files

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, nsteps=nsteps)

100%|██████████| 8/8 [07:56<00:00, 59.57s/it]


In [33]:
i = 0

# Show output files for one run.
print("Path to mdrun output files:", os.path.dirname(tpr_files[i]))
print("Output files:", os.listdir("/".join(tpr_files[i].split("/")[:-1])))
print()

# Tail log file to see performance.
tail(tpr_files[i].replace('tpr.tpr', 'md.log'), n=5)

Path to mdrun output files: /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/eq
Output files: ['tpr.tpr', 'traj.trr', 'md.log', 'confout.gro', 'state.cpt', 'ener.edr', 'mdout.mdp']

       Time:       30.071       15.036      200.0
                 (ns/day)    (hour/ns)
Performance:      287.326        0.084
Finished mdrun on rank 0 Tue Nov 15 02:01:52 2022



## 4.&nbsp; Non-Equilibrium Simulations

In [34]:
#@title 4.1&nbsp; Extract frames from equilibrium runs, and prepare TPRs

fe.equilTime = 0.  # ps to discard as equilibration
fe.bGenTiTpr = True  # Generates TPRs from extracted frames and rm GRO file.
tpr_files = fe.prepare_simulation(simType='transitions')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: transitions
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run2/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run1/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateB/run2/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run1/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateA/run2/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run1/transitions/ti1.tpr
  /content/pmxrun_1668476992/lysopath/lysozyme_benzene/protein/stateB/run2/transitions/ti1.tpr


In [35]:
#@title 4.2&nbsp; Run non-equilibrium simulations

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file, transition=True):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, transition=True)

100%|██████████| 8/8 [09:05<00:00, 68.22s/it]


In [36]:
i = 0  # tpr id

# Show output files for one run.
print("Path to mdrun output files:", os.path.dirname(tpr_files[i]))
print("Output files:", os.listdir("/".join(tpr_files[i].split("/")[:-1])))
print()

# Tail dhdl file.
print("Last 10 lines of an XVG files. The first column is time in ps, the second column is the instantaneous dH/dl.")
tail("/".join(tpr_files[i].split("/")[:-1]) + "/dhdl_1.xvg", n=10)

Path to mdrun output files: /content/pmxrun_1668476992/lysopath/lysozyme_benzene/water/stateA/run1/transitions
Output files: ['dhdl_1.xvg', 'ti1.tpr', 'mdout.mdp']

Last 10 lines of an XVG files. The first column is time in ps, the second column is the instantaneous dH/dl.
49.9820 -1.6139436
49.9840 -0.76984119
49.9860 -0.13274097
49.9880 0.24003887
49.9900 0.33624554
49.9920 0.15693951
49.9940 -0.34234333
49.9960 -1.0305147
49.9980 -1.6820192
50.0000 -2.2017336


## 5.&nbsp; Free Energy Estimation

In [37]:
# Estimate and show free energy terms.
fe.run_analysis(ligs=['lysozyme_benzene'])
fe.resultsAll

----------------
Running analysis
----------------


,dGcalc,errBoot,errAnalyt,framesA,framesB
lysozyme_benzene_correction,-32.3864,NaN,NaN,NaN,NaN
lysozyme_benzene_water_1,-4.0400,0.000000,2.680000,1.0,1.0
lysozyme_benzene_water_2,-5.0200,0.000000,5.300000,1.0,1.0
lysozyme_benzene_protein_1,22.3400,0.000000,19.970000,1.0,1.0
lysozyme_benzene_protein_2,20.0200,0.000000,5.000000,1.0,1.0
lysozyme_benzene_water,-4.5300,0.346569,3.131834,2.0,2.0
lysozyme_benzene_protein,21.1800,0.820449,9.925655,2.0,2.0


In [38]:
# Show estimated binding free energy (dGcalc).
fe.resultsSummary

,dGcalc,errAnalyt,errBoot
lysozyme_benzene,6.6764,10.408026,0.890644
